<a href="https://colab.research.google.com/github/jhermosillo/Proyecto-IA/blob/main/PROYECTO%20MIB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="font-size:50px;" align="center">Inferencia Bayesiana usando el Motor de Inferencia MIB</h1>

<h1 style="font-size:30px;" align="center">Quien escribió este cuento</h1>

In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import glob
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize


from collections import Counter, OrderedDict  #regresa un diccionario con conteos

import mib as mb
import tpproc as tp

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Cargar un archivo de excel a colab desde github
!apt-get install subversion
!svn checkout "https://github.com/jhermosillo//Proyecto-IA/trunk/Train/"
!svn checkout "https://github.com/jhermosillo//Proyecto-IA/trunk/Test/"

# Proyecto

In [2]:
archivos = glob.glob('./Train/*/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_train = tp.lee_cuentos(archivos)
df_train.head()

leyendo...
tamaño del contenido de archivos cargados:             1 KB


C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA\Proyecto-IA\tpproc.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('(','')\
C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA\Proyecto-IA\tpproc.py:85: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(')','')\


,titulo,tipo,autor,texto
0,catástrofes,minicuento,alfonso reyes,pasa hablado cierta catástrofe cósmica amenaza...
1,del perfecto gobernante,minicuento,alfonso reyes,entiende perfecto gobernante perfecto lleno pe...
2,ley profunda,minicuento,alfonso reyes,trata libertarnos simplemente enseñarnos descu...
3,los alivios,minicuento,alfonso reyes,nombres usan castellano restroom impropios bañ...
4,el buen rey león,fábula,esopo,león enojoso cruel violento sino tratable just...


In [3]:
with open("./voc_pry.txt",encoding='utf-8') as f:
    vocabulario = f.read()
vocabulario=vocabulario.split()
print(len(vocabulario))

4763


In [4]:
print(vocabulario[:10])

['vencedor', 'metió', 'apelar', 'gratitud', 'empieza', 'cirujano', 'pellizcó', 'lienzo', 'chorizo', 'mismos']


In [5]:
archivos = glob.glob('./Test/*')
archivos,nombres = tp.carga_cuentos(archivos)
df_test = tp.lee_cuentos(archivos,test=True)
df_test.head()

leyendo...
tamaño del contenido de archivos cargados:             0 KB


C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA\Proyecto-IA\tpproc.py:84: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('(','')\
C:\Users\user\Mi unidad\Clases\IA\BAYESIAN PROGRAMMING\Python_code\Proyecto-IA\Proyecto-IA\tpproc.py:85: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace(')','')\


,titulo,texto
0,divina fragilidad,dios creó universo sentía solo eternidad empez...
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...
3,el cetro,dijo rey esposa señora verdaderamente reina de...
4,el impostor,cansada esperar hombre llegó puntual sonreírme...


In [6]:
df_test['tokens'] = df_test.apply(lambda row: nltk.word_tokenize(row['texto']), axis=1)
df_test.head()

,titulo,texto,tokens
0,divina fragilidad,dios creó universo sentía solo eternidad empez...,"[dios, creó, universo, sentía, solo, eternidad..."
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...,"[cierto, día, profeta, sharía, encontró, niña,..."
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...,"[estación, blandos, céfiros, hacen, verdear, c..."
3,el cetro,dijo rey esposa señora verdaderamente reina de...,"[dijo, rey, esposa, señora, verdaderamente, re..."
4,el impostor,cansada esperar hombre llegó puntual sonreírme...,"[cansada, esperar, hombre, llegó, puntual, son..."


In [7]:
C=df_test.tokens.values
for c in C:
    print(c[:2])

['dios', 'creó']
['cierto', 'día']
['estación', 'blandos']
['dijo', 'rey']
['cansada', 'esperar']
['yendo', 'león']
['ranas', 'decidieron']
['cierto', 'día']
['noche', 'cerrada']
['ibas', 'contar']
['veces', 'vuelven']
['muerte', 'cansada']
['camión', 'detuvo']
['cierto', 'día']
['tres', 'punto']
['narciso', 'hoy']
['cierto', 'día']
['pesar', 'torpor']
['mary', 'dos']


## Preguntas

1. ¿Quién escribió "El caballo y el Lobo"?
2. "La escopeta" ¿es una fábula?
3. ¿Qué probabilidad hay de que "La muerte" haya sido escrita por José Saramago
4. ¿Pudo "La marcha del caracol" haber sido escrita por García Márquez y Fuentes en co-autoría?
5. Dado que "El águila de ala cortada y la zorra" es una fábula, ¿qué probabilidad hay de que haya sido escrita por Jalil Gibrán?
6. ¿Qué proporción hay de minicuentos?
7. ¿Qué cuentos son de Carlos Onetti?
8. ¿Quién dijo: "Desde que abrí los ojos me di cuenta que mi sitio no estaba aquí, donde yo estoy, sino en donde no estoy ni he estado nunca. En alguna parte hay un lugar vacío y ese vacío se llenará de mí y yo me asentaré en ese hueco que insensiblemente rebosará de mí, pleno de mí hasta volverse fuente o surtidor. Y mi vacío, el vacío de mí que soy ahora, se llenará de sí, pleno de sí, pleno de ser hasta los bordes."?
9. ¿Quién es el autor con más cuentos?
10. ¿Qué probablidad hay de quien haya escrito "Retiniblastoma" haya también escrito "El sol y las ranas"?

## Datos

In [8]:
df_train['Total']=df_train.texto.str.split().apply(len)

df_train.head()

,titulo,tipo,autor,texto,Total
0,catástrofes,minicuento,alfonso reyes,pasa hablado cierta catástrofe cósmica amenaza...,68
1,del perfecto gobernante,minicuento,alfonso reyes,entiende perfecto gobernante perfecto lleno pe...,120
2,ley profunda,minicuento,alfonso reyes,trata libertarnos simplemente enseñarnos descu...,35
3,los alivios,minicuento,alfonso reyes,nombres usan castellano restroom impropios bañ...,18
4,el buen rey león,fábula,esopo,león enojoso cruel violento sino tratable just...,58


## Reducción del vocabulario

### Vocabulario del conjunto de entrenamiento

In [10]:
s=df_train.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
# print(conteos)
nuevo_vocabulario = list(set([palabra for palabra in s if conteos[palabra] <= 10 and conteos[palabra]>3 ]))
print(len(nuevo_vocabulario))

7210
298


In [11]:
df_train['nuevo_texto']=df_train.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocabulario]).\
    apply(lambda x : ' '.join(x))
df_train['nuevo_total']=df_train.nuevo_texto.str.split().apply(len)
df_train['Conteos']=df_train.nuevo_texto.str.split().apply(Counter)


df_train.head()

,titulo,tipo,autor,texto,Total,nuevo_texto,nuevo_total,Conteos
0,catástrofes,minicuento,alfonso reyes,pasa hablado cierta catástrofe cósmica amenaza...,68,fondo pues corazón veces parte parte mejor vie...,9,"{'fondo': 1, 'pues': 1, 'corazón': 1, 'veces':..."
1,del perfecto gobernante,minicuento,alfonso reyes,entiende perfecto gobernante perfecto lleno pe...,120,gobernante dónde modo vivían pueblo año gobern...,29,"{'gobernante': 4, 'dónde': 1, 'modo': 1, 'viví..."
2,ley profunda,minicuento,alfonso reyes,trata libertarnos simplemente enseñarnos descu...,35,debe saber,2,"{'debe': 1, 'saber': 1}"
3,los alivios,minicuento,alfonso reyes,nombres usan castellano restroom impropios bañ...,18,vuelve,1,{'vuelve': 1}
4,el buen rey león,fábula,esopo,león enojoso cruel violento sino tratable just...,58,león sino buena bajo lobo dio paz liebre liebr...,12,"{'león': 1, 'sino': 1, 'buena': 1, 'bajo': 1, ..."


In [12]:
print(Counter(df_train.iloc[0].texto.split()))
print(Counter(df_train.iloc[0].nuevo_texto.split()))
print(df_train.iloc[0].Conteos)

Counter({'catástrofe': 3, 'parte': 2, 'pasa': 1, 'hablado': 1, 'cierta': 1, 'cósmica': 1, 'amenaza': 1, 'fondo': 1, 'universo': 1, 'ancorajes': 1, 'fragmento': 1, 'pues': 1, 'sucede': 1, 'alguna': 1, 'distante': 1, 'ignorada': 1, 'repercute': 1, 'cofrecito': 1, 'vibratorio': 1, 'corazón': 1, 'muchas': 1, 'veces': 1, 'sabemos': 1, 'rara': 1, 'inquietud': 1, 'traviesa': 1, 'dolor': 1, 'inesperado': 1, 'malestar': 1, 'llega': 1, 'ser': 1, 'físico': 1, 'corpóreo': 1, 'mejor': 1, 'reventado': 1, 'cometa': 1, 'estallado': 1, 'nebulosa': 1, 'viento': 1, 'energías': 1, 'etéreas': 1, 'desatado': 1, 'varios': 1, 'millones': 1, 'añosluz': 1, 'ahora': 1, 'llegando': 1, 'casa': 1, 'nubes': 1, 'radiantes': 1, 'visitan': 1, 'puede': 1, 'extinguirse': 1, 'estrella': 1, 'paguemos': 1, 'aún': 1, 'merecerlo': 1, 'vivimos': 1, 'morimos': 1, 'asaetados': 1, 'oscuras': 1, 'flechas': 1, 'arqueros': 1, 'sombras': 1, 'sitiados': 1})
Counter({'parte': 2, 'fondo': 1, 'pues': 1, 'corazón': 1, 'veces': 1, 'mejor':

In [13]:
print(df_train.iloc[0].texto)
print(df_train.iloc[0].nuevo_texto)

pasa hablado cierta catástrofe cósmica amenaza fondo universo catástrofe ancorajes fragmento pues sucede alguna distante ignorada catástrofe repercute cofrecito vibratorio corazón muchas veces sabemos rara inquietud traviesa parte parte dolor inesperado malestar llega ser físico corpóreo mejor reventado cometa estallado nebulosa viento energías etéreas desatado varios millones añosluz ahora llegando casa nubes radiantes visitan puede extinguirse estrella paguemos aún merecerlo vivimos morimos asaetados oscuras flechas arqueros sombras sitiados
fondo pues corazón veces parte parte mejor viento aún


#### Tamaño del vocabulario de entrenamiento

In [14]:
print(len(nuevo_vocabulario))

298


### Vocabulario del conjunto de prueba

In [16]:
s=df_test.texto.str.cat(sep=' ').split()
print(len(s))
conteos=Counter(s)
# print(conteos)
nuevo_vocab = list(set([palabra for palabra in s if conteos[palabra] <= 10 and conteos[palabra]>3 ]))
print(len(nuevo_vocab))

2224
89


In [17]:
df_test['nuevo_texto']=df_test.texto.str.split().\
    apply(lambda texto: [w for w in texto if w in nuevo_vocab]).\
    apply(lambda x : ' '.join(x))
df_test['nuevo_total']=df_test.nuevo_texto.str.split().apply(len)
df_test['Conteos']=df_test.nuevo_texto.str.split().apply(Counter)


df_test.head()

,titulo,texto,tokens,nuevo_texto,nuevo_total,Conteos
0,divina fragilidad,dios creó universo sentía solo eternidad empez...,"[dios, creó, universo, sentía, solo, eternidad...",dios sentía sentía cosa tan dios tal soledad h...,48,"{'dios': 9, 'sentía': 2, 'cosa': 1, 'tan': 2, ..."
1,dos seres iguales,cierto día profeta sharía encontró niña jardín...,"[cierto, día, profeta, sharía, encontró, niña,...",cierto profeta sharía niña niña señor profeta ...,60,"{'cierto': 1, 'profeta': 7, 'sharía': 4, 'niña..."
2,el caballo y el lobo,estación blandos céfiros hacen verdear campos ...,"[estación, blandos, céfiros, hacen, verdear, c...",vida cierto alegría así todas aquel señor aque...,11,"{'vida': 1, 'cierto': 1, 'alegría': 1, 'así': ..."
3,el cetro,dijo rey esposa señora verdaderamente reina de...,"[dijo, rey, esposa, señora, verdaderamente, re...",reina ser señor palabras cetro reina cetro mom...,19,"{'reina': 3, 'ser': 1, 'señor': 2, 'palabras':..."
4,el impostor,cansada esperar hombre llegó puntual sonreírme...,"[cansada, esperar, hombre, llegó, puntual, son...",esperar hombre primer voz jesús nunca jesús hi...,44,"{'esperar': 1, 'hombre': 2, 'primer': 1, 'voz'..."


#### Tamaño del vocabulario de prueba

In [18]:
print(len(nuevo_vocab))
print(df_test.iloc[0].texto)
print(df_test.iloc[0].nuevo_texto)

89
dios creó universo sentía solo eternidad empezó solo sentía solo necesitaba inventar cosa tan complicada universo dios contado incluso espectáculo magnífico nebulosas agujeros negros tal sentimiento soledad persistiese atormentarlo pensó pensó cabo pensar hizo mujer imagen semejanza después habiéndola hecho vio bueno tarde comprendió solo curaría definitivamente mal solo acostándose verificó aún mejor pasado algún tiempo posible saber previsión accidente biológico mente divina nació niño imagen semejanza dios niño creció convirtió joven hombre ahora bien dios pasó cabeza simple idea crear mujer dar joven sentimiento soledad afligido padre tardó repetirse hijo ahí entró diablo esperar primer impulso dios acabar ahí mismo incestuosa especie entró repente cansancio fastidio tener repetir creación hecho universo parecía tan magnífico dirá siendo dios podía hacer cuantos universos quisiese equivale desconocer naturaleza profunda dios lógicamente hecho mejor universos posibles podía hacer

#### Tamaño del vocabulario conjunto (total de palabras)

In [19]:
vocabulario = nuevo_vocabulario+nuevo_vocab
print(len(vocabulario))

387
